In [1]:
import pandas as pd

# Function for extracting data from Eurostat
def get_eurostat_datasets(dataset_name):
    csv_url = f"https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/{dataset_name}/?format=SDMX-CSV&lang=en&label=label_only"
    return pd.read_csv(csv_url)

# Get datasets
freq_education = get_eurostat_datasets('hlth_ehis_al1e')
freq_income = get_eurostat_datasets('hlth_ehis_al1i')
freq_urbanisation = get_eurostat_datasets('hlth_ehis_al1u')

# Drop unnecessary columns
for df in [freq_education, freq_income, freq_urbanisation]:
    df.drop(['freq', 'unit', 'DATAFLOW', 'LAST UPDATE'], axis=1, inplace=True)
    df.dropna(subset=['OBS_VALUE'], inplace=True)

# Remove 'Total' from specific columns in each dataset
freq_income_cleaned = freq_income[freq_income['quant_inc'] != 'Total']
freq_education_cleaned = freq_education[freq_education['isced11'] != 'All ISCED 2011 levels']
freq_urbanisation_cleaned = freq_urbanisation[freq_urbanisation['deg_urb'] != 'Total']

freq_income_cleaned.head()

C:\Users\iida.lassila\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\iida.lassila\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


,quant_inc,frequenc,sex,age,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,First quintile,Every day,Females,Total,Austria,2014,1.9,NaN
1,First quintile,Every day,Females,Total,Austria,2019,3.4,NaN
2,First quintile,Every day,Females,Total,Belgium,2014,7.7,u
3,First quintile,Every day,Females,Total,Belgium,2019,6.7,NaN
4,First quintile,Every day,Females,Total,Bulgaria,2014,2.2,NaN


In [2]:
geos_to_remove = [
    'European Union - 27 countries (from 2020)', 
    'European Union - 28 countries (2013-2020)',
    'Italy'
]

filtered_df = freq_income_cleaned[~freq_income_cleaned['geo'].isin(geos_to_remove)]

age_values_to_keep = [
    'From 15 to 24 years', 
    'From 25 to 34 years',
    'From 35 to 44 years',
    'From 45 to 54 years',
    'From 55 to 64 years', 
    'From 65 to 74 years',
    '75 years or over'
]

filtered_df = filtered_df[filtered_df['age'].isin(age_values_to_keep)]

# delete 'Never or not in the last 12 months' from column 'frequenc'
filtered_df = filtered_df[filtered_df['frequenc'] != 'Never or not in the last 12 months']

# remove 'Total' from column 'sex'
filtered_df = filtered_df[filtered_df['sex'] != 'Total']

filtered_df.head()

,quant_inc,frequenc,sex,age,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
123,First quintile,Every day,Females,From 15 to 24 years,Austria,2014,0.0,NaN
124,First quintile,Every day,Females,From 15 to 24 years,Austria,2019,0.0,NaN
125,First quintile,Every day,Females,From 15 to 24 years,Belgium,2014,0.0,u
126,First quintile,Every day,Females,From 15 to 24 years,Belgium,2019,0.0,u
127,First quintile,Every day,Females,From 15 to 24 years,Bulgaria,2014,0.8,NaN
